In [30]:
import pandas as pd
import pyarrow.feather as feather
from getEventTypes import getEventTypes

In [31]:
#parameters
root_path="C:\\Users\\bruno\\Desktop\\Diplomski\\ \\"

In [32]:
signal_path = root_path + "signal.feather"
uni_signal_path="C:\\Users\\bruno\\Desktop\\Diplomski\\uni.feather"

In [33]:
%%time
table = feather.read_table(signal_path)
signal_pyarrow = table.to_pandas()

CPU times: total: 6.03 s
Wall time: 3.69 s


In [34]:
%%time
uni_table = feather.read_table(uni_signal_path)
uni_signal_pyarrow = uni_table.to_pandas()

CPU times: total: 0 ns
Wall time: 13 ms


In [35]:
event_types = [s.replace('Snore', 'Snore_y') for s in getEventTypes()]
SleepPhases = {
    'WK' : 0,
    'N1' : 1,
    'N2' : 2,
    'N3' : 3,
    'REM' : 4
}

In [36]:
for event_type in event_types:
    if event_type not in signal_pyarrow.columns:
        signal_pyarrow[event_type] = False

In [25]:
new_patient_row_dict = {}
for event_type in event_types:
    for sleep_phase in SleepPhases:
        new_patient_row_dict[event_type + " " + sleep_phase] = [len(signal_pyarrow[(signal_pyarrow[event_type] == True) & (signal_pyarrow['Stage'] == SleepPhases[sleep_phase])])]
    
    
new_patient_row_dict['Duration'] = [len(signal_pyarrow)]
for sleep_phase in SleepPhases:
    new_patient_row_dict["Duration " + sleep_phase] = [len(signal_pyarrow[signal_pyarrow['Stage'] == SleepPhases[sleep_phase]])]

In [29]:
new_patient_row = pd.DataFrame(new_patient_row_dict)

0    6004
Name: Arousal N1, dtype: int64

In [39]:
uni_signal_pyarrow = pd.concat([uni_signal_pyarrow, new_patient_row], ignore_index=True)

In [10]:
feather.write_feather(uni_signal_pyarrow, uni_signal_path)
